In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv("~/Desktop/dataApiFast/used_car_sales.csv")

In [ ]:
zips = pd.read_csv("~/Desktop/dataApiFast/ZIP-COUNTY-FIPS_2017-06.csv")

In [ ]:
df.head()

In [ ]:
df.zipcode=df.zipcode.astype('str')
zips['ZIP']=zips['ZIP'].astype('str')

In [ ]:
df_with_state=pd.merge(df,zips[['ZIP','STATE']],left_on='zipcode',right_on='ZIP',how='inner')
df_with_state

In [ ]:
zips[['ZIP','STATE']][zips['STATE']=='NY']

In [ ]:
zips.groupby(['ZIP','STATE'], as_index=False).first()[['ZIP','STATE']]